# version of proxyFinder algorithm using neural network to make predictions

In [1]:
import numpy as np
import pandas as pd
import sys
import statsmodels.api as sm
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

In [4]:
# Setting some hyperparameters and making sure we have a GPU
%matplotlib inline
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import random
import tqdm
import os
from torchviz import make_dot

# Set the device to use
device = torch.device("cpu")

# If you'd like to use a GPU instead of a CPU to run the following code, you can uncomment the following line
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Hyperparameters
num_epochs = 10
batch_size = 256

## Fixing Random Seed for Reproducibility
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)


# This line prints out the type of device you will run the notebook in.
print(str(device))

c:\Users\kirin\anaconda3\Lib\site-packages\torch\nested\_internal\nested_tensor.py:417: UserWarning: Failed to initialize NumPy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.11 from "c:\Users\kirin\anaconda3\python.exe"
  * The NumPy version is: "1.24.3"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: The specified module could not be found.
 (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_numpy.cpp:84.)
  values=torch.randn(3, 3, d

cpu


In [6]:
def proxy_finder_validate(item, candidates, df1, df2, predictors, orthogonal_vars):

    # validate proxies and st item
    assert item in df1.columns, f'AssertionError: item {item} not in df1.columns'

    assert predictors, f'AssertionError: missing predictors'
    
    for c in predictors:
        assert c in df1.columns, f'AssertionError: predictor {c} not in df1.columns'
        assert c in df2.columns, f'AssertionError: predictor {c} not in df2.columns' # only because we need same variable in second dataset        
    
    for c in candidates:
        assert c in df2.columns, f'AssertionError: candidate {c} not in df2.columns'
        
    if (orthogonal_vars != None):
        for c in orthogonal_vars:
            assert c in df2.columns, f'AssertionError: orthogonal variable {c} not in df2.columns'
                

In [7]:
# rescale all columns to be between 0 and 1, inclusive. Drop any non-numeric columns.
def data_rescale(df):
   
    # Select only the numeric columns
    numeric_cols = df.select_dtypes(include=['number']).columns
    
    # Initialize the scaler
    scaler = MinMaxScaler()

    # Fit the scaler to the data and transform it
    scaled_values = scaler.fit_transform(df[numeric_cols])

    # Create a new DataFrame with the scaled values, maintaining the original column names
    scaled_df = pd.DataFrame(scaled_values, columns=numeric_cols, index=df.index)
    
    return scaled_df

In [ ]:
# return a trained neural network to predict df[item] using df[predictors_df1]
# report error and crash if predictors don't predict item
def get_model(predictors_df1, item, test_pct, df):

    # create linear model such that item_predicted = B0 + B1X1 + ... + BnXn
    prepped_data = df[predictors_df1 + [item]].dropna(axis=0)
    X = prepped_data[predictors_df1].to_numpy()

    y = prepped_data[item].to_numpy()


    #split into train and test data

    test_size = (int) (X.shape[0] * test_pct)
    train_size = (int) (X.shape[0] * (1-test_pct))

    x_train = X[:-train_size]
    x_test = X[-test_size:]

    # Split the targets into training/testing sets
    y_train = y[:-train_size]
    y_test = y[-test_size:]


    # run linear regression
  
    
    if (mean_squared_error(regr.predict(x_test), y_test) > 0.02):
        print('Input Error: Predictors cannot predict item in df1')
        print('Aborting program')
        sys.exit(-1)
    
    return regr